<a href="https://colab.research.google.com/github/gangdung/DeepLearningBasics/blob/master/Naver_Movie_Review_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import pandas as pd
import numpy as np
import matplot as mp
import matplotlib
#matplotlib.use('agg')  #matplotlib는 GUI 이벤트 루프에 연결되어 예기치 않은 동작을 일으키는 "tkagg"백엔드에 액세스, 일반 "agg"백엔드는 GUI에 전혀 연결되지 않습니다.
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

print('-------------------------------------------------------------------------------')
print('-------------------------------------------------------------------------------')
print('-------------------------------------------------------------------------------')

train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력
print('상위 5개 출력(train_data) : ', train_data[:5])  # 상위 5개 출력
print('상위 5개 출력(test_data)  : ', test_data[:5])  # 상위 5개 출력

# 데이타검증
print(train_data['document'].nunique(), train_data['label'].nunique()) #document열에서 중복을 제거한 샘플의 개수
train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
print('총 샘플의 수(null제거전):',len(train_data))

# 박스플롯 띄우기
#train_data['label'].value_counts().plot(kind = 'bar')
#plt.show()

print('label건수비교 : ' , train_data.groupby('label').size().reset_index(name = 'count'))
print('Null값Check : ' , train_data.isnull().values.any())

print(train_data.isnull().sum())
print(train_data.loc[train_data.document.isnull()])

train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print('Null값Check : ' , train_data.isnull().values.any()) # Null 값이 존재하는지 확인

print('총 샘플의 수(null제거후):',len(train_data))

#데이터 전처리
text = 'do!!! you expect... people~ to~ read~ the FAQ, etc. and actually accept hard~! atheism?@@'
print(re.sub(r'[^a-zA-Z ]', '', text)) #알파벳과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
print(train_data[:5])

train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())
print(train_data.loc[train_data.document.isnull()][:5])

train_data = train_data.dropna(how = 'any')
print('총 샘플의 수(null+한글외문자제거후):',len(train_data))

test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))
print('-------------------------------------------------------------------------------')


stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()
print('Test:', okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True))
print('-------------------------------------------------------------------------------')


stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
X_train = []
for sentence in train_data['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

print('상위 3개의 샘플만 출력(X_train) : ', X_train[:3])

X_test = []
for sentence in test_data['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

print('상위 3개의 샘플만 출력(X_test)  : ', X_test[:3])
print('-------------------------------------------------------------------------------')


tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
print('각 단어에 고유한 정수가 부여 : ', tokenizer.word_index)
print('-------------------------------------------------------------------------------')


threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

# 정수 인코딩 과정
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print('정수 인코딩 :', X_train[:3])

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

#####5.빈 샘플(empty samples) 제거
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))
print('-------------------------------------------------------------------------------')

#####6.패딩
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
#plt.hist([len(s) for s in X_train], bins=50)
#plt.xlabel('length of samples')
#plt.ylabel('number of samples')
#plt.show()
print('-------------------------------------------------------------------------------')


def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 30
below_threshold_len(max_len, X_train)

X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

print('상위 3개의 샘플만 출력(X_train) : ', X_train[:3])
print('상위 3개의 샘플만 출력(X_test)  : ', X_test[:3])
print('-------------------------------------------------------------------------------')
print('-------------------------------------------------------------------------------')
print('-------------------------------------------------------------------------------')



from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

loaded_model_naver = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model_naver.evaluate(X_test, y_test)[1]))

def sentiment_predict(old_sentence):
  new_sentence = okt.morphs(old_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model_naver.predict(pad_new)) # 예측
  if(score > 0.5):
    print(old_sentence, ' : ', "{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print(old_sentence, ' : ', "{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))


sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')
sentiment_predict('이 영화 핵노잼 ㅠㅠ')
sentiment_predict('이딴게 영화냐 ㅉㅉ')
sentiment_predict('감독 뭐하는 놈이냐?')
sentiment_predict('와 개쩐다 정말 세계관 최강자들의 영화다')

print('-------------------------------------------------------------------------------')
print('-------------------------------------------------------------------------------')
print('-------------------------------------------------------------------------------')





